In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '1'

import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from hopkins import Hopkins
from tqdm import tqdm


In [ ]:
# 获取当前脚本文件所在目录的绝对路径
current_dir = os.getcwd()
# 构造 "data" 目录的路径
data_dir = os.path.join(current_dir, "../data")  # 返回上一级目录下的 "data" 目录
# 定义文件名
file_name = f"output_state_1vs1.txt"
file_path = os.path.join(data_dir, file_name)

In [ ]:
# 读取文本文件
with open(file_path, 'r') as file:
    lines = file.readlines()

# 解析每行数据
data = []
for line in lines:
    row = list(map(float, line.strip().split()))
    data.append(row)

# 将数据转换为 NumPy 数组
import numpy as np
data_array = np.array(data)

print("Data shape:", data_array.shape)
print("Sample of data:")
print(data_array[:5])  # 打印前5行数据

In [ ]:
labels = data_array[:, -1:].tolist()  # 提取最后两列作为标签，并转换为列表
# 创建字典来存储数据分组
grouped_data = {}

In [ ]:
for i, label in enumerate(labels):
    label_key = tuple(label)
    if label_key not in grouped_data:
        grouped_data[label_key] = []
    grouped_data[label_key].append(data_array[i, :-1])  # 去除最后两列
# 打印每个分组中的数据数量
for label, group in grouped_data.items():
    print(f"Label {label}: {len(group)} samples")

In [ ]:
for label, group in grouped_data.items():
    grouped_data[label] = np.array(group)

In [ ]:
target_label = (1.0, )
if target_label in grouped_data:
    cluster_data = grouped_data[target_label]
    Hopkins_value = Hopkins(cluster_data)
    print(Hopkins_value)
else:
    Hopkins_value = 0.0

In [ ]:
from HybridClustering import hybrid_clustering
if(Hopkins_value>0.55):
    kmeans_labels = hybrid_clustering(grouped_data[target_label],2,25)
    from WriteCluster import write_cluster_data, write_dbscan_cluster_data
    write_cluster_data(grouped_data[target_label], kmeans_labels, "kmeans")


In [ ]:
from boundaries import extract_boundaries
upper_bounds, lower_bounds = extract_boundaries('cluster_2_kmeans.txt')
print("\n")
print("kmeans Upper boundaries:", upper_bounds)
print("kmeans Lower boundaries:", lower_bounds)

In [ ]:
# 使用 t-SNE 进行降维，将数据投影到二维空间
tsne = TSNE(n_components=2, random_state=42)
with tqdm(total=100, desc="TSNE") as pbar:
    X_embedded = tsne.fit_transform(cluster_data)
    pbar.update(100)

# 绘制聚类结果的二维散点图
plt.figure(figsize=(9, 6))

# 绘制每个簇的数据点
for cluster_label in np.unique(kmeans_labels):
    visual_data = X_embedded[kmeans_labels == cluster_label]
    plt.scatter(visual_data[:, 0], visual_data[:, 1], label=f'Cluster {cluster_label}')

plt.title('t-SNE Visualization of kmeans Clustered Data in 2D')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend()
plt.show()